# Traffic Sign Recognition using CNN

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import torch.utils.data as data
from torchinfo import summary
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import time

## Data Preparation

In [ ]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])

data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((80, 80)), 
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ]),
    
    'val': transforms.Compose([
    transforms.Resize((80, 80)), 
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ]),
    
    'test': transforms.Compose([
    transforms.Resize((80, 80)), 
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])
} #resizing and normalizing images

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 5
num_epochs = 15
#hyperparameters

data_dir = 'TrafficSignDataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}
#loading data

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

class_names = image_datasets['train'].classes


## Display Images

In [ ]:
def imshow(inp):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    plt.imshow(inp)
    plt.title( 'Samples of Training Images')
    plt.show()

inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

print('All Classes: ', class_names)

imshow(out)

## Network Model
### (CNN to FCNN)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 9) 
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.conv3 = nn.Conv2d(10, 16, 3)
        self.conv4 = nn.Conv2d(16, 20, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(20 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 5)

    def forward(self, x):
        # CNN layer
        x = self.conv1(x) # 80*80*3 -> 72*72*6      
        x = F.relu(x)
        x = self.pool(x) # -> 36*36*6
        x = self.conv2(x) # -> 32*32*10
        x = F.relu(x)
        x = self.pool(x) # ->16*16*10
        x = self.conv3(x) # -> 14*14*16
        x = F.relu(x)
        x = self.pool(x) # -> 7*7*16
        x = self.conv4(x) # -> 5*5*20
        x = F.relu(x)
        
        # FCNN layer
        x = x.view(-1, 20 * 5 * 5) # flatten
        x = self.fc1(x) # 500(5*5*20) -> 120
        x = F.relu(x)
        x = self.fc2(x) # 120 -> 80
        x = F.relu(x)
        x = self.fc3(x) # 80 -> 5 classes
        
        return x


## Training Loop

In [ ]:
def train_model(model, optimizer, criterion, num_epochs):
    since = time.time()
    best_acc = 0.0
    
    print("Training phase:")
    print()
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()
                
            total_loss = 0.0
            corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
            
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                total_loss += loss.item()
                corrects += (preds == labels).sum()
                
            epoch_average_loss = total_loss / (dataset_sizes[phase]/batch_size)
            epoch_acc = corrects / dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_average_loss:.4f} Acc:{100 * epoch_acc:.4f}%')
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc

        print()
        
    print(f'Best val Acc: {100 * best_acc:4f}%')
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed/60:.0f}min and {time_elapsed%60:.0f}s')
    
    return model

### Calling the training model and loop

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = train_model(model, optimizer, criterion, num_epochs)

## Testing

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(dataset_sizes['test'])]
    n_class_samples = [0 for i in range(dataset_sizes['test'])]
    for images, labels in dataloaders['test']:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum()
        for i in range (batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = 100.0 * n_correct.item()/n_samples
    print("Testing phase:")
    print()
    print('============================================')
    print(f"Accuracy of the network: {acc}%")
    print('============================================')
    print()
    print('--------------------------------------------')
    
    for i in range(len(class_names)):
        acc = 100.0 * n_class_correct[i]/n_class_samples[i]
        print(f"Accuracy of {class_names[i]}: {acc}%")
        print('--------------------------------------------')

#### Change this cell to code to save your weights

PATH = './tsr_wts.pth'
torch.save(model.state_dict(), PATH)